In [0]:
%pip install databricks-labs-dqx==0.9.2 

In [0]:
dbutils.library.restartPython()

In [0]:
%run /Workspace/Users/glamero17@gmail.com/test_dq/utils/custom_rules_library_py

In [0]:
%run /Workspace/Users/glamero17@gmail.com/test_dq/utils/dq_utils

In [0]:
# MAGIC %md
# MAGIC # 🚀 Motor de Calidad de Datos (Orquestador)
# MAGIC 
# MAGIC Este notebook ejecuta el framework de calidad de datos
# MAGIC 
# MAGIC **Actualizaciones v10:**
# MAGIC * **Arquitectura Normalizada:** Este script ahora hace un JOIN entre
# MAGIC   `dq_validations_catalog` y `dq_rule_library` para obtener el
# MAGIC   `technical_rule_name` y el `rule_type` en tiempo de ejecución.
# MAGIC * Lee de la tabla `table_config` (nuevo nombre).
# MAGIC * Sigue usando el motor de API 2 (`check_funcs.py`).
# MAGIC * Mantiene la lógica de `def main()` y particionamiento por fecha.

# COMMAND ----------

# DBTITLE 1, 1. Imports y Widgets
import yaml
import uuid
import time
import json
import os
from datetime import datetime, date
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, lit, current_timestamp
from delta.tables import DeltaTable

# --- Funciones disponibles en check_funcs.py ---
from databricks.labs.dqx.check_funcs import (
    is_not_null, is_unique, regex_match, is_in_range, is_in_list, compare_datasets,
    sql_expression, is_aggr_not_greater_than, foreign_key, is_aggr_not_less_than, is_valid_timestamp,
    is_aggr_equal, is_aggr_not_equal, is_not_empty, is_not_null_and_not_empty, is_not_equal_to,
    is_valid_date, is_not_in_future, sql_query, is_not_null_and_is_in_list, is_not_in_range,
    is_not_less_than, is_not_greater_than, is_equal_to
)

#no me ha dejado importar:  has_valid_schema


# --- Widgets ---
#dbutils.widgets.text("table_id", "", "Id de la tabla a validar")
dbutils.widgets.text("severity_filter", "", "Filtro de severidad (Opcional)")
dbutils.widgets.text("validation_id_filter", "", "Filtro para 1 validación específica (Opcional)")
dbutils.widgets.text("table_name", "maestro_demo", "Id de la tabla a validar")

# COMMAND ----------

# DBTITLE 2, 2. Carga de librerías y definición de constantes/mapas

dbutils.widgets.text("catalog_name", "workspace", "Catálogo de UC donde residen las tablas")
dbutils.widgets.text("schema_name", "dq_framework", "Esquema de UC donde residen las tablas")

CATALOG = dbutils.widgets.get("catalog_name")
SCHEMA = dbutils.widgets.get("schema_name")

EXECUTION_TABLE = f"{CATALOG}.{SCHEMA}.dq_execution_traceability"
VALIDATIONS_TABLE = f"{CATALOG}.{SCHEMA}.dq_validations_traceability"

TABLE_CONFIG = f"{CATALOG}.{SCHEMA}.dq_tables_config"
RULE_LIB_TABLE = f"{CATALOG}.{SCHEMA}.dq_rules_library"
CATALOG_TABLE = f"{CATALOG}.{SCHEMA}.dq_validations_catalog"

# --- Carga de librerías de funciones custom ---
# ¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿ como hacer en PRO (en repor???????????????????
#las funciones van a carpetas en repos (ej.
#/Repos/usuario/proyecto/utils/dq_utils.py
#/Repos/usuario/proyecto/utils/custom_rules_library_py.py)
#Luego se importa como cualquier libreria. Pej.
# from utils.dq_utils import alguna_funcion
#from utils.custom_rules_library_py import otra_funcion

# Mapea el string del nombre técncio con un nombre de función disponible
BUILT_IN_RULE_FUNCTIONS = {
    "is_not_null": is_not_null, "is_not_empty": is_not_empty, "is_not_null_and_not_empty": is_not_null_and_not_empty,
    "is_in_list": is_in_list, "regex_match": regex_match, "is_in_range": is_in_range, "is_not_equal_to": is_not_equal_to,
    "sql_expression": sql_expression, "is_valid_date": is_valid_date, "is_not_in_future": is_not_in_future, 
    "is_not_null_and_is_in_list": is_not_null_and_is_in_list,"is_not_less_than": is_not_less_than, "is_not_in_range": is_not_in_range,
    "is_not_greater_than": is_not_less_than, "is_equal_to": is_equal_to,"is_valid_timestamp": is_valid_timestamp,
    "is_unique": is_unique, "is_aggr_not_greater_than": is_aggr_not_greater_than, "is_aggr_not_less_than": is_aggr_not_less_than, 
    "is_aggr_equal": is_aggr_equal, "is_aggr_not_equal": is_aggr_not_equal, "foreign_key": foreign_key, 
    "sql_query": sql_query, "compare_datasets": compare_datasets
}
   
ROW_LEVEL_FUNCS = [
    "is_not_null", "is_not_empty", "is_not_null_and_not_empty", "is_in_list", 
    "regex_match", "is_in_range", "sql_expression", "is_valid_date", "is_not_in_future",
    "is_valid_timestamp", "is_not_null_and_is_in_list", "is_not_in_range",
    "is_not_less_than", "is_not_greater_than", "is_equal_to", "is_not_equal_to"
]
DATASET_LEVEL_FUNCS = [
    "is_unique", "is_aggr_not_greater_than", "is_aggr_not_less_than", 
    "is_aggr_equal", "is_aggr_not_equal", "foreign_key", "sql_query",
    "compare_datasets"
]

CUSTOM_RULE_FUNCTIONS = {
    "custom_validate_dni": custom_validate_dni 
}

# COMMAND ----------

# DBTITLE 3, 3. Función Principal (main)
def main():
    """
    Flujo principal de orquestación.
    """    
    # --- 1. Inicializar variables de trazabilidad ---
    start_time = time.time()
    execution_id = str(uuid.uuid4())
    exec_timestamp = datetime.now()
    exec_date = exec_timestamp.date()
    
    status = "FAILED"
    total_records = 0
    validations_executed_count = 0
    validations_failed_count = 0
    table_name = "N/A"
    table_id_param = "N/A"
    df = None
    
        # --- 2. Cargar parámetros del Job ---
#--------------------------------------------- esta linea siguiente borrar porque debería venir del JOB
    
    table_name_param = dbutils.widgets.get("table_name")
    severity_param = dbutils.widgets.get("severity_filter")
    validation_id_param = dbutils.widgets.get("validation_id_filter")

    if not table_name_param:
        print("Error Crítico: 'table_name' no puede estar vacío.")
    
    # --- 3. Leer la configuración mMaestra ---
    
    config_row = (spark.table(TABLE_CONFIG)
                    .filter(col("table_name") == table_name_param)
                    .first())
    
    if config_row is None:
        print(f"No se encontró configuración en {TABLE_CONFIG} para la tabla '{table_name_param}'")
    
    table_id_param = config_row.table_id
    table_name = config_row.table_name
    table_to_validate = config_row.table_name_tech
    primary_key = config_row.primary_key
    staging_evidences_table = config_row.staging_evidences_table
    
    # --- 4. Log de inicio ---
    log_execution_start(execution_id, exec_timestamp, table_id_param, EXECUTION_TABLE)
    
    # --- 5. Leer las validaciones a ejecutar ---
    filter_query = f"table_id = '{table_id_param}' AND is_active = True"
    if severity_param:
        filter_query += f" AND severity = '{severity_param}'"
    if validation_id_param:
        filter_query += f" AND validation_id = '{validation_id_param}'"
    
    # Leer las validaciones y cruzarlas con la librería para obtener más información
    df_validations_to_run = (spark.table(CATALOG_TABLE)
                                .filter(filter_query))
    
    df_library = (spark.table(RULE_LIB_TABLE)
                    .select("rule_id", "technical_rule_name", col("implementation_type").alias("rule_type")))

    rules_list = (df_validations_to_run
                    .join(df_library, "rule_id", "left")
                    .select(
                        df_validations_to_run.validation_id,
                        df_validations_to_run.rule_id,
                        df_validations_to_run.validation_definition,
                        df_library.rule_type,
                        df_library.technical_rule_name
                    )
                    .collect())
    
    if not rules_list:
        print(f"AVISO: No se encontraron validaciones activas para el filtro: {filter_query}")
        status = "SUCCESS"
        duration = round(time.time() - start_time, 2)
        return (execution_id, status, duration, validations_executed_count, validations_failed_count)
        
    print(f"Se encontraron {len(rules_list)} validaciones para ejecutar.")
    

    # --- 6. Carga de datos ---
    #!!!!!!!!!!!!!!!!aqui no se permite la cache
    #df = spark.table(table_to_validate).cache()
    df = spark.table(table_to_validate)
    total_records = df.count()
    print(f"Se han cargado {total_records} registros.")

    # --- 7. Motor de ejecución de validaciones ---
    print(f"--- Iniciando la ejecución de {len(rules_list)} validaciones ---")

    # Reglas que requieren un literal de comparación pero también permiten una columna
    RULES_REQUIRING_LITERAL_CONVERSION = [
        "is_equal_to", "is_not_equal_to", 
        "is_not_less_than", "is_not_greater_than"
    ]

    for validation in rules_list:
        
        validation_id = validation.validation_id
        rule_id = validation.rule_id
        validation_definition_str = validation.validation_definition
        rule_type = validation.rule_type
        technical_rule_name = validation.technical_rule_name
        
        print(f"\n--- Ejecutando validación ID: {validation_id}, Regla: {technical_rule_name}) ---")
        validations_executed_count += 1
        
        df_failed = None
        rule_status = "PASSED"
        
        try:
            if not rule_type or not technical_rule_name:
                raise Exception(f"No se pudo encontrar la regla (rule_id: {rule_id}) en la dq_rule_library.")
                
            params = json.loads(validation_definition_str)

            failed_field = primary_key
            if "column" in params:
                failed_field = params["column"]
            elif "columns" in params and params["columns"]:
                failed_field = params["columns"][0]

            #cambio a literal del parametro value o limit
            if technical_rule_name in RULES_REQUIRING_LITERAL_CONVERSION:
                if "value" in params and isinstance(params["value"], str):
                    # Si el valor es un string, lo convertimos en un Literal de Spark
                    params["value"] = lit(params["value"])
                if "limit" in params and isinstance(params["limit"], str):
                    # Si el límite es un string, lo convertimos en un Literal de Spark
                    params["limit"] = lit(params["limit"])


            # --- CASO 1: REGLA BUILT-IN ---
            if rule_type == 'BUILT-IN':
                if technical_rule_name not in BUILT_IN_RULE_FUNCTIONS:
                    rule_status = "ERROR"
                    print(f"  > ERROR: Regla BUILT-IN desconocida: {technical_rule_name}")
                else:
                    func_to_call = BUILT_IN_RULE_FUNCTIONS[technical_rule_name]
                    if technical_rule_name in ROW_LEVEL_FUNCS:
                        params_exec = params.copy()
                        if 'column' in params_exec: params_exec['column'] = col(params_exec['column'])
                        condition_col = func_to_call(**params)
                        #!!!!!!!!!En serverless compute no es posible
                        #df_failed = df.filter(condition_col.isNotNull()).cache()
                        df_failed = df.filter(condition_col.isNotNull())
                    elif technical_rule_name in DATASET_LEVEL_FUNCS:
                        (condition_col, apply_func) = func_to_call(**params)
                        df_with_cond = apply_func(df, spark, {}) 
                        #!!!!!!!!!En serverless compute no es posible
                        #df_failed = df_with_cond.filter(condition_col.isNotNull()).cache()
                        df_failed = df_with_cond.filter(condition_col.isNotNull())

            # --- CASO 2: REGLA CUSTOM (Python) ---
            elif rule_type == 'CUSTOM':
                if technical_rule_name in CUSTOM_RULE_FUNCTIONS:
                    function_to_call = CUSTOM_RULE_FUNCTIONS[technical_rule_name]
                    #!!!!!!!!!En serverless compute no es posible
                    #df_failed = function_to_call(df, params.get("columns")).cache()
                    df_failed = function_to_call(df, params.get("columns"))
                else:
                    rule_status = "ERROR"
                    print(f"  > ERROR: Función CUSTOM desconocida: {technical_rule_name}")
            
            # --- CASO 3: REGLA CUSTOM (SQL) ---
            elif rule_type == 'CUSTOM_SQL_UDF':
                udf_name = technical_rule_name
                udf_columns = params.get("columns", [])
                if udf_name and udf_columns:
                    sql_expression = f"{udf_name}({', '.join(udf_columns)})"
                    #!!!!!!!!!En serverless compute no es posible
                    #df_failed = df.filter(f"NOT ({sql_expression})").cache()
                    df_failed = df.filter(f"NOT ({sql_expression})")
                else:
                    rule_status = "ERROR"
                    print(f"  > ERROR: CUSTOM_SQL_UDF mal configurada.")
            
            # --- CASO 4: Tipo desconocido ---
            else:
                rule_status = "ERROR"
                print(f"  > ERROR: Tipo de regla '{rule_type}' desconocido.")
            
            # --- 7a. Procesamiento de resultados (por regla) ---
            failed_count = 0
            if df_failed is not None and not df_failed.isEmpty():
                rule_status = "FAILED"
                validations_failed_count += 1
                failed_count = df_failed.count()
              
                log_evidences_staging(
                    df_failed=df_failed, 
                    staging_table_name=staging_evidences_table, 
                    execution_id=execution_id, 
                    exec_date=exec_date,
                    validation_id=validation_id, 
                    primary_key_col=primary_key,
                    failed_field=failed_field,
                    catalog=CATALOG,
                    schema=SCHEMA
                )
                #!!!!!!!!!En serverless compute no es posible
                #df_failed.unpersist()
            
            # --- 7b. Log de trazabilidad de validación ---
            log_validations_traceability(
                execution_id=execution_id,
                exec_date=exec_date,
                validation_id=validation_id,
                rule_id=rule_id,
                status=rule_status,
                perimeter=total_records,
                failed_count=failed_count,
                validations_table=VALIDATIONS_TABLE
            )

        except Exception as e:
            print(f"  > Erro crítico ejecutando regla: {technical_rule_name} (Id: {validation_id}). Error: {e}")
            validations_failed_count += 1
            log_validations_traceability(
                execution_id=execution_id,
                exec_date=exec_date,
                validation_id=validation_id,
                rule_id=rule_id,
                status="ERROR",
                perimeter=total_records,
                failed_count=-1,
                validations_table=VALIDATIONS_TABLE
            )
            #!!!!!!!!!!!!!!!!!!! en sereveless no es posible
            #if df_failed:
                #df_failed.unpersist()
    
    # --- Fin del Bucle ---
    print("\n--- Ejecución de validaciones finalizada ---")
    status = "SUCCESS" if validations_failed_count == 0 else "FAILED"
    
        # --- 8. Finalización y log de fin de ejecución ---
    #if df is not None:
        #!!!!!!!!!!!!!!!esto no se permite en serveless compute
        #df.unpersist()
    
    end_time = time.time()
    duration = round(end_time - start_time, 2)
    
    log_execution_finish(
        execution_id=execution_id,
        status=status,
        duration=duration,
        validations_exec=validations_executed_count,
        validations_fail=validations_failed_count,
        trace_table=table_name
    )
    
    print(f"Framework finalizado para '{table_name}' en {duration}s. Estado final: {status}")
    return execution_id, status

# COMMAND ----------

# DBTITLE 4, 4. Punto de Entrada de Ejecución
if __name__ == "__main__":
    print("antes main")
    result = main()
    exec_id, status = result[0], result[1]
    #butils.notebook.exit(f"Ejecución {exec_id} completada. Estado: {status}")
        
    #print(f"Fallo fatal en la ejecución: {e}")
    #dbutils.notebook.exit(f"Fallo fatal en la ejecución: {e}")